In [1]:
import numpy as np
from data import Vertebral_column
from data import Co_Author
from Processing_Data import Abanole
from Processing_Data import Ecoli
from Processing_Data import Ecloli1
from Processing_Data import Ecoli3
from Processing_Data import Glass1
from Processing_Data import Glass4
from Processing_Data import Haberman
from Processing_Data import Waveform
from Processing_Data import New_thyroid2
from Processing_Data import Page_blocks
from Processing_Data import Pima_Indians_Diabetes
from Processing_Data import Satimage
from Processing_Data import Transfusion
from Processing_Data import Yeast
from Processing_Data import Haberman_All
from Processing_Data import Transfusion_All
from Processing_Data import PimaIndians_All
from data import indian_liver_patient
#from data import spect_heart
from wsvm.application import Wsvm
from svm.application import Svm
from sklearn.svm import SVC
#from sklearn.metrics import f1_score
from sklearn.metrics  import classification_report,precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import _safe_indexing
from sklearn import metrics
import math
from datetime import datetime
from fuzzy.weight import fuzzy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from Processing_Data import Ecoli_Kfold
from Processing_Data import Haberman_KFold
from Processing_Data import Transfution_Kfold

(array([[-0.20356986, -0.23616822, -0.01535438, ..., -0.10297836,
        -0.00333545,  1.9115189 ],
       [-0.71319985, -0.07417361,  0.33070229, ..., -0.10297836,
        -1.03616576, -0.16111872],
       [-0.42198271,  0.16881831, -1.05352439, ..., -0.10297836,
         0.16880293, -0.53796193],
       ...,
       [ 0.59727728,  0.08782101,  0.67675896, ..., -0.10297836,
        -2.75754961, -0.53796193],
       [-0.13076557, -0.39816283,  1.48422452, ..., -0.10297836,
         1.37377162, -0.53796193],
       [ 0.014843  ,  0.08782101,  0.79211118, ..., -0.10297836,
        -0.34761222, -0.53796193]]), array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,

In [2]:
def svm_lib(X_train, y_train,X_test):
    svc=SVC(probability=True, kernel='linear')
    model = svc.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [3]:
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [4]:
def svm(C,X_train, y_train,X_test):
    model = Svm(C)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [5]:
def is_tomek(X,y, class_type):
    print(y)
    print(type(y))
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1]
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample:
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample)
                X_dangxet.append(nn_index[index_sample])
                links[index_sample] = True

    return links,X_dangxet,X_tl

In [6]:
def Gmean(y_test,y_pred):
    cm_WSVM = metrics.confusion_matrix(y_test, y_pred)
    sensitivity = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    specificity = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(sensitivity*specificity)
    return specificity,sensitivity,gmean

In [7]:
def metr(X_train,y_test,test_pred,se,sp,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    print("So luong samples: ",len(X_train))
    print("\n",classification_report(y_test, test_pred))
    print("SP      : ",sp)
    print("SE      : ",se)
    print("Gmean   : ",gmean)
    print("F1 Score: ",f1_score(y_test, test_pred))
    print("Accuracy: ",accuracy_score(y_test,test_pred))
    print("AUC     : ",roc_auc_score(y_test, test_pred))
    print("Ma tran nham lan: \n",confusion_matrix(y_test, test_pred))
    

In [8]:
def metr_text(f,X_train,y_test,test_pred,sp,se,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    f.write(f"\n\nSo luong samples Tong: {len(X_train)+len(y_test)}")
    f.write(f"\n\nSo luong samples training: {len(X_train)}")
    f.write(f"\nSo luong samples testing: {len(y_test)}\n")
    f.write("\n"+str(classification_report(y_test, test_pred)))
    f.write(f"\nSP      : {sp:0.4f}")
    f.write(f"\nSE      : {se:0.4f}")
    f.write(f"\nGmean   : {gmean:0.4f}")
    f.write(f"\nF1 Score: {f1_score(y_test, test_pred):0.4f}")
    f.write(f"\nAccuracy: {accuracy_score(y_test,test_pred):0.4f}")
    f.write(f"\nAUC     : {roc_auc_score(y_test, test_pred):0.4f}")
    f.write("\n\nMa tran nham lan: \n"+str(confusion_matrix(y_test, test_pred)))

In [9]:
def compute_weight(X, y,name_method ="actual_hyper_lin", name_function = "exp", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            d = method.own_class_center_divided(X, y)
        elif name_method == "distance_center_own_opposite_tam":
            d_own, d_opp, d_tam = method.distance_center_own_opposite_tam(X,y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
        elif name_function == "func_own_opp_new":
            W = function.func_own_opp_new(d_own,d_opp,pos_index,neg_index,d_tam)
    except Exception as e:
        print('dont exist function')
        print(e)
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [10]:
def fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction):
    if namemethod =="own_class_center_opposite" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_center)
        f.write(f"\n\t Beta 'own_class_center_opposite' with exp = {beta_center}\n")
    elif namemethod =="own_class_center" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
        f.write(f"\n\t Beta 'own_class_center' with exp = {beta_center}\n")
    elif namemethod =="own_class_center_divided" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
        f.write(f"\n\t Beta 'own_class_center_divided' with exp = {beta_center}\n")
    elif namemethod =="estimated_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
        f.write(f"\n\t Beta 'estimated_hyper_lin' with exp = {beta_estimate}\n")
    elif namemethod =="actual_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_actual)
        f.write(f"\n\t Beta 'actual_hyper_lin' with exp = {beta_actual}\n")
    else:   
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction)
    return distribution_weight

In [11]:
y = np.array([5,6,7,8,9,10])
a = np.where(y>5)
a

(array([1, 2, 3, 4, 5], dtype=int64),)

In [12]:
def data_tomelinks(f,C,weight,X_test,y_test,X_train,y_train,n_neighbors,clf=None,namemethod=None,namefunction=None):
    links,xdx,xtl = is_tomek(X_train,y_train,class_type=[-1.0])
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,weight)
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    specificity,sensitivity,gmean = Gmean(y_test,y_predict)
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)
    y_nn = []
    neg_pred = clf.predict(X_train[neg_index])
    idx_neg_wrong = np.where(neg_pred != -1.0)
    
    new_W[idx_neg_wrong] = 0

    for ind,i in enumerate(xdx):
        y_pred = clf.predict([X_train[i]])  #
        if y_pred == -1.0:                          
            knn_X = (nn2.kneighbors([X_train[i]])[1]).tolist() 
            for j in knn_X[0]:
                y_nn.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn
        else:
            new_W[xtl[ind]] = new_W[xtl[ind]]/2
            new_W[i] = new_W[i]*1.25

    y_nn = np.array(y_nn)
    if len(y_nn)>0:
        y_nn = np.array_split(y_nn, len(y_nn)/n_neighbors)
    for ind,i in enumerate(range(0,len(y_nn))):      #
        if 1 not in y_nn[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => xóa X_train[i]
            new_W[xdx[ind]] = 0
        else:
            new_W[xtl[ind]] = new_W[xtl[ind]]/2
            new_W[xdx[ind]] = new_W[xdx[ind]]*1.25

    return new_W,gmean,sensitivity

In [13]:
def lfb(f,C,weight,namemethod,namefunction,T,X_test,y_test,X_train,y_train,n_neighbors,thamso1,thamso2): #loop find the best
    gmax = 0
    tmax = 0
    semax = 0
    t_semax = 0
    for i in range(0,T):
        f.write(f"\t\t Vong Lap thu: T = {i+1}")
        f.write(f"\n===================================================================================================================")
        f.write(f"\n\n\tFuzzy SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
        weight, gmeanfirst, SEfirst = data_tomelinks(f,C,weight,X_test,y_test,X_train,y_train,n_neighbors,clf=None,namemethod=namemethod,namefunction=namefunction)
        #distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
        clf = Wsvm(C,weight)
        #print(X_train)
        #print(y_train)
        clf.fit(X_train, y_train)
        #print(clf.predict([X_test_val[5]]))
        pred2 = clf.predict(X_test)    #X_val
        sp,se,gmean = Gmean(y_test,pred2) #y_val

        pred_all = clf.predict(X_test)    #X_val
        sp_all,se_all,gmean_all = Gmean(y_test,pred_all)

        if(gmeanfirst > gmax):
            gmax = gmeanfirst
            tmax = i+100
        if (gmean_all > gmax):
            gmax = gmean_all
            tmax = i
        if (SEfirst > semax):
            semax = SEfirst
            t_semax = i+100
        if (se_all > semax):
            semax = se_all
            t_semax = i   
        #metr(X_train,y_test_val,pred2,sp,se,gmean)
        f.write(f"\n\n\t****** Danh gia tren tap Validation:\n")
        metr_text(f,X_train,y_test,pred2,sp,se,gmean)
        f.write(f"\n\n\t****** Danh gia tren tap Test:\n")
        metr_text(f,X_train,y_test,pred_all,sp_all,se_all,gmean_all)
        # if ((gmeanfirst - gmean) <= thamso1) or (gmeanfirst > thamso2):
        #     f.write("\n_____Gmean_ERROR!!!____\n")
        #     print("\n_____Gmean_ERROR!!!____\n")
        #     return X_train, y_train
        # else:
        #     gmean = gmeanfirst
        f.write(f"\n===================================================================================================================\n")
    f.write(f"\nFuzzy SVM name_method = '{namemethod}',name_function = '{namefunction}'")
    f.write(f"\n*** T = {tmax}; K = {n_neighbors}; GmeanMax = {gmax:0.4f}\n")
    f.write(f"\n*** T = {t_semax}; K = {n_neighbors}; SeMax = {semax:0.4f}\n")
    f.write(f"\n===================================================================================================================\n")
    return weight

In [14]:
C = 100
thamso1 = 1
thamso2 = 1
T = 1
n_neighbor = 5
test_size = [0.2,0.3,0.4]
testsize_val = 0.2
K_big = 5
K_small = 5
# data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
#             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]
data = [Transfution_Kfold]

# !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!
beta_center, beta_estimate, beta_actual = 0.5, 0.8, 1 # !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!
# !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!

name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
#name_method =["own_class_center_divided"]
name_function = ["lin_center_own","exp","func_own_opp_new","lin"]

time = datetime.now().strftime("%d%m%Y_%H%M%S")
filepath = "./text_script"

#svc lib
svc = SVC(probability=True, kernel='linear')
#svm scratch
svm_scr = Svm(C)
#W.svm

for dataset in data:
    filename = (str(dataset).split("\\")[-1]).split(".")[0]
    f = open(f"{filepath}/Data_{filename}_{time}_Detail.txt", "w")
    f.write(f"\nC = {C}, thamso1 = {thamso1}, thamso2 = {thamso2}, T = {T}, n_neighbors = {n_neighbor}  \n")
    f.write(f"\n\n\tUSING DATASET : {filename}\n")

    f2 = open(f"{filepath}/Data_{filename}_{time}_Main.txt", "w")
    f2.write(f"\nC = {C}, thamso1 = {thamso1}, thamso2 = {thamso2}, T = {T}, n_neighbors = {n_neighbor}  \n")
    f2.write(f"\n\n\tUSING DATASET : {filename}\n")
    

    #print(f"\n\tUSING DATASET : {filename}\n")
    # for testsize in test_size:
    #     X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize)
    X, y = dataset.load_data()
    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
    for train_index, test_index in kfold_validation.split(X,y):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]
        #Scalling Data
        sc_X = StandardScaler()
        X_train = sc_X.fit_transform(X_train)
        X_test = sc_X.transform(X_test)
        y_train = np.array(y_train)
        print(type(X_train), type(y_train))

        #print(f"\t======== TestSize: {testsize} ========")
        #print("So luong sample nguyen ban ban dau: ",len(X_train)+len(X_test_val)+len(X_test))

        # f.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
        f.write("\n\n\t====== NOT USING TOMEKLINKS ========== \n")

        # f2.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
        f2.write("\n\n\t====== NOT USING TOMEKLINKS ========== \n")

        #Svm library
        f.write("\n\n\tSVM LIBRARY starting...\n")
        f2.write("\n\n\tSVM LIBRARY starting...\n")
        #print("SVM LIBRARY starting...\n")
        test_pred = svm_lib(X_train, y_train,X_test)
        sp,se,gmean = Gmean(y_test,test_pred)
        #metr(X_train,y_test,test_pred,sp,se,gmean)
        metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
        metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

        # #Svm scratch
        # f.write("\n\n\tSVM starting...\n")
        # f2.write("\n\n\tSVM starting...\n")
        # print("SVM starting...\n")
        # test_pred = svm(C,X_train, y_train,X_test)
        # sp,se,gmean = Gmean(y_test,test_pred)
        # #metr(X_train,y_test,test_pred,sp,se,gmean)
        # metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
        # metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

        #Wsvm
        f.write("\n\n\tW.SVM starting...\n")
        f2.write("\n\n\tW.SVM starting...\n")
        print("W.SVM starting...\n")
        N, d = X_train.shape
        distribution_weight = np.ones(N)
        test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
        sp,se,gmean = Gmean(y_test,test_pred)
        #metr(X_train,y_test,test_pred,sp,se,gmean)
        metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
        metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

        #FuzyyWsvm
        for namemethod in name_method:
            for namefunction in name_function:
                if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                    continue
                elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                    continue
                elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                    continue
                elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                    continue
                elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                    continue
                elif namemethod == "distance_center_own_opposite_tam" and namefunction == "lin":
                    continue
                elif namemethod == "own_class_center" and namefunction == "lin":
                    continue
                elif namemethod == "estimated_hyper_lin" and namefunction == "lin":
                    continue
                elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
                    continue
                else:
                    f.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    f2.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                    __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                    test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
                    sp,se,gmean = Gmean(y_test,test_pred)
                    metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
                    metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

        # f.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
        f.write("\n\n\t====== USING TOMEKLINKS ========== \n")

        # f2.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
        f2.write("\n\n\t====== USING TOMEKLINKS ========== \n")

        # #Svm library
        # f.write("\n\n\tSVM LIBRARY starting...\n")
        # f2.write("\n\n\tSVM LIBRARY starting...\n")
        # #print("SVM LIBRARY starting...\n")
        # test_pred = svm_lib(X_train, y_train,X_test)
        # sp,se,gmean = Gmean(y_test,test_pred)
        # #metr(X_train,y_test,test_pred,sp,se,gmean)
        # metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
        # metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

        
        # #Wsvm
        # f.write("\n\n\tW.SVM starting...\n")
        # f2.write("\n\n\tW.SVM starting...\n")
        # print("W.SVM starting...\n")
        # N, d = X_train.shape
        # distribution_weight = np.ones(N)
        # test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
        # sp,se,gmean = Gmean(y_test,test_pred)
        # #metr(X_train,y_test,test_pred,sp,se,gmean)
        # metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
        # metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)


        #FuzyyWsvm
        for namemethod in name_method:
            for namefunction in name_function:
                if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                    continue
                elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                    continue
                elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                    continue
                elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                    continue
                elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                    continue
                elif namemethod == "distance_center_own_opposite_tam" and namefunction == "lin":
                    continue
                elif namemethod == "own_class_center" and namefunction == "lin":
                    continue
                elif namemethod == "estimated_hyper_lin" and namefunction == "lin":
                    continue
                elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
                    continue
                else:
                    f.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    f2.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                    __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                    new_W = lfb(f,C,distribution_weight,namemethod,namefunction,T,X_test,y_test,X_train,y_train,n_neighbor,thamso1,thamso2)
                    test_pred = wsvm(C,X_train, y_train, X_test, new_W)
                    sp,se,gmean = Gmean(y_test,test_pred)

                    #metr(X_train,y_test,test_pred,sp,se,gmean)
                    metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
                    metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

    f.write("\n===================================================================================\n")
    f.close()
    f2.write("\n===================================================================================\n")
    f2.close()

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
W.SVM starting...



KeyboardInterrupt: 

In [ ]:
f.close()
f2.close()